### Question 1 ###

In [9]:
#importing libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
# getting data and setting beautifulsoup
webseitdata=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(webseitdata,'html.parser')

In [12]:
# create lists to store data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [13]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if (len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text)

In [14]:
postalCodeList=list(map(lambda x:x.rstrip("\n"),postalCodeList))
boroughList=list(map(lambda x:x.rstrip("\n"),boroughList))
neighborhoodList=list(map(lambda x:x.rstrip("\n"),neighborhoodList))

In [15]:
df=pd.DataFrame({"PostalCode":postalCodeList, "Borough":boroughList,"Neighborhood":neighborhoodList})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- Processing table to only rows with boroughs

In [16]:
df_clean=df[df.Borough!="Not assigned"].reset_index(drop=True)
df_clean.head()

- Merging rows with the same postal code and two different boroughs

In [17]:
df_clean = df_clean.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_clean.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


- for rows with not assigned neighbours then the borough name was assigned as neighbour

In [18]:
df_clean["Neighborhood"]=df_clean["Neighborhood"].replace("Not assigned",df_clean["Borough"])
df_clean.head()